# Visualización de Faros por Región

Este notebook permite visualizar los faros de una región específica utilizando la API de Overpass y folium para la visualización del mapa.

En los notebooks podemos incluir la instalación de las dependencias necesarias, como por ejemplo en este caso con pip.

In [ ]:
# Instalación de dependencias necesarias
%pip install overpass folium pandas

## Importación de Librerías
- **overpass**: Para acceder a datos de OpenStreetMap
- **folium**: Para crear mapas interactivos
- **pandas**: Para manejo de datos
- **math**: Para cálculos matemáticos
- **IPython.display**: Para mostrar el mapa en el notebook

In [3]:
import overpass
import folium
from folium import plugins
import pandas as pd
from IPython.display import display
import math

## Obtención de Datos de Faros
La función `get_lighthouses()`:
- Recibe el nombre de una región
- Construye una consulta Overpass que:
  - Busca el área administrativa de nivel 4 (región)
  - Encuentra todos los faros (nodos, vías y relaciones)
  - Obtiene su geometría completa
- Maneja errores y timeout de la API

In [4]:
def get_lighthouses(region):
    """Obtiene los faros de una región específica usando Overpass API."""
    api = overpass.API(timeout=25)
    
    # Consulta para encontrar faros en la región especificada
    query = f"""
    area["admin_level"="4"]["name"="{region}"]->.searchArea;
    (
      node["man_made"="lighthouse"](area.searchArea);
      way["man_made"="lighthouse"](area.searchArea);
      relation["man_made"="lighthouse"](area.searchArea);
    );
    out geom;
    """
    
    try:
        result = api.get(query)
        print(f"Encontrados {len(result.get('features', []))} elementos")
        return result
    except Exception as e:
        print(f"Error al obtener datos: {e}")
        return None

## Procesamiento de Coordenadas
La función `get_coordinates_from_feature()`:
- Extrae coordenadas de elementos GeoJSON
- Convierte del formato [lon, lat] a [lat, lon]
- Valida que el elemento sea un punto

In [5]:
def get_coordinates_from_feature(feature):
    """Extrae las coordenadas de un feature GeoJSON."""
    if feature.get('geometry', {}).get('type') == 'Point':
        # Las coordenadas en GeoJSON están en formato [lon, lat]
        lon, lat = feature['geometry']['coordinates']
        return lat, lon
    return None

## Generación de Popups
La función `generate_popup()`:
- Crea el contenido HTML para los popups
- Incluye información detallada:
  - Nombre del faro
  - Nombre alternativo
  - Características técnicas
  - Enlaces a Wikidata/Wikipedia

In [6]:
def generate_popup(properties):
    """Genera el texto del popup basado en las propiedades de un faro."""
    # Extraer el nombre y usar un valor predeterminado si falta
    name = properties.get('name', 'Faro sin nombre')
    popup_text = f"<b>{name}</b><br>"

    # Añadir detalles específicos del faro si están presentes
    if 'alt_name' in properties:
        popup_text += f"Nombre alternativo: {properties['alt_name']}<br>"
    if 'seamark:light:character' in properties:
        popup_text += f"Característica: {properties['seamark:light:character']}<br>"
    if 'seamark:light:colour' in properties:
        popup_text += f"Color: {properties['seamark:light:colour']}<br>"
    if 'seamark:light:range' in properties:
        popup_text += f"Alcance: {properties['seamark:light:range']} millas náuticas<br>"
    if 'seamark:light:height' in properties:
        popup_text += f"Altura: {properties['seamark:light:height']} m<br>"
    if 'seamark:light:period' in properties:
        popup_text += f"Período: {properties['seamark:light:period']} s<br>"
    if 'seamark:light:sequence' in properties:
        popup_text += f"Secuencia: {properties['seamark:light:sequence']}<br>"
    if 'wikidata' in properties:
        popup_text += (
            f"<a href='https://www.wikidata.org/wiki/{properties['wikidata']}' target='_blank'>"
            "Ver en Wikidata</a><br>"
        )
    if 'wikipedia' in properties:
        popup_text += (
            f"<a href='https://{properties['wikipedia']}' target='_blank'>"
            "Ver en Wikipedia</a><br>"
        )
    
    return popup_text


## Creación del Mapa Interactivo
La función `create_lighthouse_map()` es el núcleo de la visualización:

### Características principales:
1. **Marcadores Dinámicos**:
   - El halo varía según el alcance del faro
   - Color amarillo para simular la luz
   - Tamaño proporcional al alcance

2. **Información Detallada**:
   - Popups con datos técnicos
   - Enlaces a recursos externos
   - Estado del faro

3. **Estilo del Mapa**:
   - Tema oscuro (CartoDB Dark_Matter)
   - Escala adaptativa
   - Zoom automático a la región

In [7]:
import folium

def create_lighthouse_map(region):
    """Crea un mapa interactivo con marcadores personalizados cuyo halo varía según el alcance del faro."""
    try:
        # Obtener los datos de los faros (reemplazar con tus datos reales)
        lighthouses = get_lighthouses(region)  # Simula una llamada para obtener el JSON
        if not lighthouses or 'features' not in lighthouses:
            print(f"No se encontraron faros en {region}")
            return

        # Extraer las coordenadas válidas
        coordinates = [
            feature['geometry']['coordinates'][::-1]  # Invertir lat/lon
            for feature in lighthouses['features']
            if feature['geometry']['type'] == 'Point'
        ]
        
        if not coordinates:
            print("No se encontraron coordenadas válidas")
            return

        # Encontrar los valores máximo y mínimo de 'range' para normalizar el halo
        ranges = []
        for feature in lighthouses['features']:
            range_str = feature['properties'].get('seamark:light:range', '0')
            try:
                range_value = float(range_str)
                ranges.append(range_value)
            except ValueError:
                continue
        
        if not ranges:
            print("No se encontraron valores válidos para 'range'")
            return
        
        max_range = max(ranges)
        min_range = min(ranges)

        # Crear el mapa con un tema oscuro
        m = folium.Map(
            location=[sum(c[0] for c in coordinates) / len(coordinates),
                      sum(c[1] for c in coordinates) / len(coordinates)],
            zoom_start=9,
            tiles="CartoDB Dark_Matter"
        )

        # Añadir los faros al mapa con marcadores personalizados
        for feature in lighthouses['features']:
            if feature['geometry']['type'] == 'Point':
                coords = feature['geometry']['coordinates'][::-1]  # Invertir lat/lon
                properties = feature.get('properties', {})
                popup_text = generate_popup(properties)
                
                # Obtener el valor de 'range' del faro y convertirlo a un número (si es posible)
                range_str = properties.get('seamark:light:range', '0')
                try:
                    range_value = float(range_str)  # Intentamos convertir el valor de 'range' a float
                except ValueError:
                    range_value = 0  # Si no se puede convertir, asignamos un valor por defecto (0)

                # Normalizar el halo en función del 'range'
                if range_value > 0:
                    # Mapeamos el 'range' entre un rango de 15 a 100 para el halo (más pequeño)
                    halo_radius = 15 + ((range_value - min_range) / (max_range - min_range)) * 85
                else:
                    halo_radius = 15  # Si no tiene un valor de 'range', un halo mínimo
                
                # Crear el marcador luminoso con un círculo pequeño
                icon_html = f"""
                <div style="
                    background-color: rgba(255, 255, 0, 0.5);
                    border-radius: 50%;
                    width: 10px;  # Tamaño base pequeño
                    height: 10px;  # Tamaño base pequeño
                    position: relative;
                    box-shadow: 0 0 {halo_radius}px {halo_radius / 3}px rgba(255, 255, 0, 0.7);
                    ">
                </div>
                """
                
                # Crear el marcador con un ícono personalizado
                folium.Marker(
                    location=coords,
                    icon=folium.DivIcon(html=icon_html),
                    popup=folium.Popup(popup_text, max_width=300)
                ).add_to(m)

        return m
    except Exception as e:
        print(f"Error al crear el mapa: {e}")


## Ejemplo de Uso
Demostración práctica:
- Se puede especificar cualquier región (ej: "Galicia", "Catalunya", "Andalucía")
- El mapa se ajusta automáticamente a la región seleccionada
- Muestra todos los faros disponibles en OpenStreetMap

### Notas:
- La calidad de los datos depende de OpenStreetMap
- Algunas regiones pueden tener datos más completos que otras
- El tiempo de respuesta depende de la API de Overpass

In [ ]:
# Ejemplo con Galicia
region = "Galicia"
mapa = create_lighthouse_map(region)
if mapa:
    display(mapa)

## Referencias y Recursos
- [Documentación de Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API)
- [Documentación de Folium](https://python-visualization.github.io/folium/)
- [OpenStreetMap Wiki - Lighthouse Tag](https://wiki.openstreetmap.org/wiki/Tag:man_made%3Dlighthouse)